In [1]:
"""
TODO - BUild a model for Superset using DNN
"""


'\nTODO - BUild a model for Superset using DNN\n'

In [2]:
import pandas as pd
import numpy as np

Uploading Dataset

In [3]:
super=pd.read_csv('datasets/superset.csv')

In [4]:
super.columns

Index(['label', 'statement'], dtype='object')

In [5]:
super_x=super.statement
super_y=super.label

In [6]:
super_x.shape

(233413,)

Train test Split


In [7]:
#train test split to form test from megaset
from sklearn.model_selection import train_test_split

X_train_temp, sup_test_x, y_train_temp, sup_test_y = train_test_split(super_x,super_y,test_size=0.2,random_state=12)
print(X_train_temp.shape)
print(sup_test_x.shape)

# mega_test_x : Final testing file with statements
# mega_test_y : Final testing file with ratings

sup_train_x, sup_dev_x, sup_train_y, sup_dev_y = train_test_split(X_train_temp,y_train_temp,test_size=46683,random_state=12)

print(sup_train_x.shape)
print(sup_dev_x.shape)

# mega_train_x : Training file with statements
# mega_train_y : Training file with ratings
# mega_dev_x   : Development testing file with statements
# mega_dev_y   : Development testing file with ratings
print(type(sup_train_x))
print(sup_train_y.shape)
print("Final size of train/dev/test : ",sup_train_x.shape,"/",sup_dev_x.shape,"/",sup_test_x.shape)

(186730,)
(46683,)
(140047,)
(46683,)
<class 'pandas.core.series.Series'>
(140047,)
Final size of train/dev/test :  (140047,) / (46683,) / (46683,)


In [8]:
#convert y from true -1 and false -0
sup_train_y.replace({'true': 1, 'fake': 0},inplace=True)
sup_test_y.replace({'true': 1, 'fake': 0},inplace=True)
sup_dev_y.replace({'true': 1, 'fake': 0},inplace=True)


In [9]:
import tensorflow
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.models import Sequential
from keras import layers
import time
from keras.models import load_model
from keras.layers import Dropout
from keras.regularizers import l2


Using TensorFlow backend.


In [12]:
tokenizer = Tokenizer()#this is the dictionary that is built , no memory error here
tokenizer.fit_on_texts(sup_train_x)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
print('vocab size ',vocab_size)  
    
    


vocab size  597387


Tokenize and pad the attribute

In [10]:
def tokpad(train_x):
    train_text = tokenizer.texts_to_sequences(train_x)
    train_padtext = pad_sequences(train_text, padding='post', maxlen=144708)
    return train_padtext

In [13]:
#Model 2
embedding_dim = 25
h1=int(2/3*embedding_dim)
h2=int(2/3*h1)
print(h1,h2)

model2 = Sequential()
model2.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=144708))
model2.add(layers.GlobalMaxPool1D())
model2.add(layers.Dense(h1, activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model2.add(layers.Dense(h2, activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model2.summary()

16 10
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 144708, 25)        14934675  
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                416       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 14,935,272
Trainable params: 14,935,272
Non-trainable params: 0
_________________________________________________________________


In [107]:
"""""""""
#run this to reset the model
model2.save("model.h5")
model = load_model('model.h5')

"""""""""

''

In [15]:
def trainmodel(train_x,train_y):
    print('loading model')
    model=load_model('model.h5')
    print('model loaded')
    print('Running tokpad')
    trainpad=tokpad(train_x)
    print('Tokpad is done')
    print('Start training')
    
    history = model.fit(trainpad,train_y,
                    epochs=20,
                    verbose=1,
                    batch_size=10)
    print('Fitting is done')
    print('saving the model')
    model.save("model.h5")
    print('model is saved')
    print("Cleaning Ram")
    gc.collect()

In [98]:
#first 10 000 only
trainmodel(sup_train_x[:10000],sup_train_y[:10000])

loading model
model loaded
Running tokpad
Tokpad is done
Start training
Epoch 1/25
10000/10000 [==============================] - 504s 50ms/step - loss: 0.4488 - accuracy: 0.7578
Epoch 2/25
10000/10000 [==============================] - 503s 50ms/step - loss: 0.2507 - accuracy: 0.8935
Epoch 3/25
10000/10000 [==============================] - 502s 50ms/step - loss: 0.1759 - accuracy: 0.9328
Epoch 4/25
10000/10000 [==============================] - 501s 50ms/step - loss: 0.1269 - accuracy: 0.9502
Epoch 5/25
10000/10000 [==============================] - 501s 50ms/step - loss: 0.0950 - accuracy: 0.9664
Epoch 6/25
10000/10000 [==============================] - 501s 50ms/step - loss: 0.0747 - accuracy: 0.9739
Epoch 7/25
10000/10000 [==============================] - 501s 50ms/step - loss: 0.0606 - accuracy: 0.9778
Epoch 8/25
10000/10000 [==============================] - 501s 50ms/step - loss: 0.0497 - accuracy: 0.9828
Epoch 9/25
10000/10000 [==============================] - 502s 50ms/step

In [103]:
#saving the weights
temp_weights = [layer.get_weights() for layer in model.layers]
#added the regularizers

In [111]:
#setting the weights back
for i in range(len(temp_weights)):
    model.layers[i].set_weights(temp_weights[i])

In [117]:
#models back and updated
#training the next 10000
trainmodel(sup_train_x[10000:20000],sup_train_y[10000:20000])

loading model
model loaded
Running tokpad
Tokpad is done
Start training
Epoch 1/20
10000/10000 [==============================] - 494s 49ms/step - loss: 0.6157 - accuracy: 0.7135
Epoch 2/20
10000/10000 [==============================] - 506s 51ms/step - loss: 0.3804 - accuracy: 0.8698
Epoch 3/20
10000/10000 [==============================] - 512s 51ms/step - loss: 0.3088 - accuracy: 0.9112
Epoch 4/20
10000/10000 [==============================] - 501s 50ms/step - loss: 0.2575 - accuracy: 0.9336
Epoch 5/20
10000/10000 [==============================] - 500s 50ms/step - loss: 0.2198 - accuracy: 0.9497
Epoch 6/20
10000/10000 [==============================] - 502s 50ms/step - loss: 0.1906 - accuracy: 0.9603
Epoch 7/20
10000/10000 [==============================] - 501s 50ms/step - loss: 0.1686 - accuracy: 0.9655
Epoch 8/20
10000/10000 [==============================] - 501s 50ms/step - loss: 0.1505 - accuracy: 0.9711
Epoch 9/20
10000/10000 [==============================] - 501s 50ms/step

In [ ]:
trainmodel(sup_train_x[20000:30000],sup_train_y[20000:30000])

loading model
model loaded
Running tokpad
Tokpad is done
Start training
Epoch 1/20
 2010/10000 [=====>........................] - ETA: 7:02 - loss: 0.3647 - accuracy: 0.8532